In [1]:
from __future__ import print_function
import os
import neat

import pandas as pd
import numpy as np
import random

import torch
import torch.nn as nn
import torch.optim as optim


from explaneat.core.backprop import NeatNet
from explaneat.core import backprop
from explaneat.core.backproppop import BackpropPopulation
# from explaneat.visualization import visualize
from explaneat.core.experiment import ExperimentReporter
from explaneat.core.utility import one_hot_encode


from sklearn import datasets, metrics
from sklearn.preprocessing import StandardScaler, normalize, OneHotEncoder
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

from copy import deepcopy

import time
from datetime import datetime

In [2]:
RANDOM_SEED      = 42


USE_CUDA = torch.cuda.is_available()
USE_CUDA = False
device = torch.device("cuda:1" if USE_CUDA else "cpu")



In [3]:

def xor(a, b, threshold = 0.5):
    response = False
    if a > threshold and b < threshold:
        response = True
    if a < threshold and b > threshold:
        response = True
    # return (1.0, 0.0) if response else (0.0, 1.0)
    return 1.0 if response else 0.0
    

def create_n_points(n, size, min=0.0, max=1.0):
    data = []
    for _ in range(n):
        data.append([
            random.uniform(min, max) for ii in range(size)
        ])

    return data

# correct solution:
def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0) # only difference

def overUnder(val, threshold):
    return 1. if val > threshold else 0

xor_inputs = create_n_points(400, 2, -1, 1)

xor_outputs = [
    [xor(tup[0], tup[1], 0)] for tup in xor_inputs
]

In [4]:
xor_inputs

[[-0.30219806184575515, -0.5787113808187629],
 [-0.6521530259960249, 0.6228622896619378],
 [0.9493806296642389, 0.527968228308308],
 [-0.7750667363719195, -0.6240764198590985],
 [-0.7863379440285565, 0.8563829429289656],
 [-0.818581703360477, 0.2748178443325302],
 [-0.41431049653814367, -0.9985117410070952],
 [-0.3582203478521624, 0.36017036349730813],
 [0.6087276731390971, 0.9678966549161461],
 [0.32207628695936985, -0.0917705962997113],
 [0.43820542029190057, -0.4516281926324901],
 [-0.1821507959061981, -0.6506485204199222],
 [0.12698965850352129, 0.533511390576745],
 [-0.42986523996707526, 0.8862033096592672],
 [0.8265782294687527, 0.918251840010037],
 [-0.6759307010992033, -0.5281267895512471],
 [0.5274652093070702, -0.9805885064828719],
 [0.24171932921816985, 0.8483737569259862],
 [-0.6453498114106087, 0.9645156850572383],
 [-0.14881163839424438, 0.23620461392940761],
 [0.053557351249525764, 0.651765091253002],
 [0.6292128066482405, -0.7531209772393406],
 [0.14682035665539628, 0.7

In [5]:
xor_outputs

[[0.0],
 [1.0],
 [0.0],
 [0.0],
 [1.0],
 [1.0],
 [0.0],
 [1.0],
 [0.0],
 [1.0],
 [1.0],
 [0.0],
 [0.0],
 [1.0],
 [0.0],
 [0.0],
 [1.0],
 [0.0],
 [1.0],
 [1.0],
 [0.0],
 [1.0],
 [0.0],
 [0.0],
 [1.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [1.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [1.0],
 [0.0],
 [0.0],
 [1.0],
 [0.0],
 [0.0],
 [0.0],
 [1.0],
 [0.0],
 [1.0],
 [1.0],
 [1.0],
 [1.0],
 [1.0],
 [0.0],
 [0.0],
 [1.0],
 [1.0],
 [1.0],
 [1.0],
 [0.0],
 [0.0],
 [1.0],
 [0.0],
 [0.0],
 [1.0],
 [0.0],
 [1.0],
 [0.0],
 [0.0],
 [1.0],
 [0.0],
 [1.0],
 [1.0],
 [1.0],
 [0.0],
 [0.0],
 [0.0],
 [1.0],
 [1.0],
 [0.0],
 [1.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [1.0],
 [0.0],
 [0.0],
 [0.0],
 [1.0],
 [1.0],
 [1.0],
 [1.0],
 [1.0],
 [1.0],
 [1.0],
 [1.0],
 [1.0],
 [0.0],
 [1.0],
 [0.0],
 [0.0],
 [0.0],
 [1.0],
 [1.0],
 [0.0],
 [1.0],
 [0.0],
 [0.0],
 [1.0],
 [1.0],
 [0.0],
 [1.0],
 [1.0],
 [1.0],
 [0.0],
 [1.0],
 [1.0],
 [0.0],
 [1.0],
 [1.0],
 [1.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],


# Set up run


In [6]:
config_path = "./config_xor"
base_config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                     neat.DefaultSpeciesSet, neat.DefaultStagnation,
                     config_path)

In [7]:
def instantiate_population(config, xs, ys, saveLocation):

    if not os.path.exists(saveLocation):
        os.makedirs(saveLocation)
        
    config.save(os.path.join(saveLocation, 'config.conf'))

    # Create the population, which is the top-level object for a NEAT run.
    p = BackpropPopulation(config, 
                            xs, 
                            ys, 
                            criterion=nn.BCEWithLogitsLoss())

    # Add a stdout reporter to show progress in the terminal.
    p.add_reporter(neat.StdOutReporter(True))
    stats = neat.StatisticsReporter()
    p.add_reporter(stats)
    p.add_reporter(neat.Checkpointer(5, filename_prefix=str(saveLocation) + "checkpoint-" ))
    bpReporter = backprop.BackpropReporter(True)
    p.add_reporter(bpReporter)
    p.add_reporter(ExperimentReporter(saveLocation))
    
    return p

In [10]:
def eval_genomes(genomes, config):
    
    print(genomes)
    loss = nn.BCELoss()
    loss = loss.to(device)
    for genome_id, genome in genomes.items():
        net = neat.nn.FeedForwardNetwork.create(genome, config)
        preds = []
        for xi in xor_inputs:
            preds.append(net.activate(xi))
        genome.fitness = float(1./loss(torch.tensor(preds).to(device), torch.tensor(xor_outputs)))

In [15]:
config = base_config
saveLocation = './'
maxNGenerations = 20
p = instantiate_population(config, xor_inputs, xor_outputs, saveLocation)
# Run for up to nGenerations generations.
winner = p.run(eval_genomes, maxNGenerations, nEpochs = 5)

g = p.best_genome


The function - generationStart - has just started at 1595137275.252438

 ****** Running generation 0 ****** 

The function - generationStart - took 0.0004019737243652344 seconds to complete
The function - pre_backprop - has just started at 1595137275.252865
The function - pre_backprop - took 6.29425048828125e-05 seconds to complete
The function - backprop - has just started at 1595137275.2529511
about to start backprop with 5 epochs
mean improvement: 0.0
best improvement: tensor(0., grad_fn=<SubBackward0>)
best loss: tensor(0.6820, grad_fn=<DivBackward0>)
The function - backprop - took 7.294240951538086 seconds to complete
The function - post_backprop - has just started at 1595137282.548584
The function - post_backprop - took 0.0053060054779052734 seconds to complete
The function - evaluate fitness - has just started at 1595137282.555732
{1: <neat.genome.DefaultGenome object at 0x7fe1f7bfd760>, 2: <neat.genome.DefaultGenome object at 0x7fe1fb4a3ee0>, 3: <neat.genome.DefaultGenome objec

mean improvement: 0.0
best improvement: tensor(0., grad_fn=<SubBackward0>)
best loss: tensor(0.7216, grad_fn=<DivBackward0>)
The function - backprop - took 6.408343076705933 seconds to complete
The function - post_backprop - has just started at 1595137303.3802838
The function - post_backprop - took 5.125999450683594e-05 seconds to complete
The function - evaluate fitness - has just started at 1595137303.380373
{8: <neat.genome.DefaultGenome object at 0x7fe1fb499cd0>, 6: <neat.genome.DefaultGenome object at 0x7fe1fb401fa0>, 12: <neat.genome.DefaultGenome object at 0x7fe1fb406ca0>, 13: <neat.genome.DefaultGenome object at 0x7fe1fb406550>, 14: <neat.genome.DefaultGenome object at 0x7fe1fb4a45b0>}
The function - evaluate fitness - took 0.015513181686401367 seconds to complete
The function - post evaluate - has just started at 1595137303.396172
Population's average fitness: 0.65144 stdev: 0.36352
Best fitness: 1.11606 - size: (1, 1) - species 1 - id 12
Key: 12
Fitness: 1.1160575151443481
No

mean improvement: 0.0
best improvement: tensor(0., grad_fn=<SubBackward0>)
best loss: tensor(0.7117, grad_fn=<DivBackward0>)
The function - backprop - took 5.936420917510986 seconds to complete
The function - post_backprop - has just started at 1595137321.384664
The function - post_backprop - took 3.1948089599609375e-05 seconds to complete
The function - evaluate fitness - has just started at 1595137321.3847291
{12: <neat.genome.DefaultGenome object at 0x7fe1fb406ca0>, 8: <neat.genome.DefaultGenome object at 0x7fe1fb499cd0>, 21: <neat.genome.DefaultGenome object at 0x7fe1fb406550>, 22: <neat.genome.DefaultGenome object at 0x7fe1f7bcddf0>, 23: <neat.genome.DefaultGenome object at 0x7fe1fb4c0c10>}
The function - evaluate fitness - took 0.010432958602905273 seconds to complete
The function - post evaluate - has just started at 1595137321.3952472
Population's average fitness: 0.51258 stdev: 0.41411
Best fitness: 1.11606 - size: (1, 1) - species 1 - id 12
Key: 12
Fitness: 1.1160575151443481

mean improvement: 0.0
best improvement: tensor(0., grad_fn=<SubBackward0>)
best loss: tensor(0.7130, grad_fn=<DivBackward0>)
The function - backprop - took 6.933720111846924 seconds to complete
The function - post_backprop - has just started at 1595137341.270141
The function - post_backprop - took 5.030632019042969e-05 seconds to complete
The function - evaluate fitness - has just started at 1595137341.270247
{26: <neat.genome.DefaultGenome object at 0x7fe1fb4a9ee0>, 24: <neat.genome.DefaultGenome object at 0x7fe1fb48b190>, 30: <neat.genome.DefaultGenome object at 0x7fe1fb48b3d0>, 31: <neat.genome.DefaultGenome object at 0x7fe1fb4a9070>, 32: <neat.genome.DefaultGenome object at 0x7fe1fb4176d0>}
The function - evaluate fitness - took 0.014997005462646484 seconds to complete
The function - post evaluate - has just started at 1595137341.285381
Population's average fitness: 0.77011 stdev: 0.48689
Best fitness: 1.35044 - size: (2, 2) - species 1 - id 26
Key: 26
Fitness: 1.3504399061203003
N

mean improvement: 0.0
best improvement: tensor(0., grad_fn=<SubBackward0>)
best loss: tensor(0.6934, grad_fn=<DivBackward0>)
The function - backprop - took 9.675577878952026 seconds to complete
The function - post_backprop - has just started at 1595137366.287292
The function - post_backprop - took 4.792213439941406e-05 seconds to complete
The function - evaluate fitness - has just started at 1595137366.2873752
{26: <neat.genome.DefaultGenome object at 0x7fe1fb4a9ee0>, 34: <neat.genome.DefaultGenome object at 0x7fe1fb41ca00>, 39: <neat.genome.DefaultGenome object at 0x7fe1fb41cd60>, 40: <neat.genome.DefaultGenome object at 0x7fe1fb4c07c0>, 41: <neat.genome.DefaultGenome object at 0x7fe1fb4221c0>}
The function - evaluate fitness - took 0.016423940658569336 seconds to complete
The function - post evaluate - has just started at 1595137366.303926
Population's average fitness: 1.01457 stdev: 0.41738
Best fitness: 1.38097 - size: (2, 2) - species 1 - id 40
Key: 40
Fitness: 1.3809747695922852


mean improvement: 0.0
best improvement: tensor(0., grad_fn=<SubBackward0>)
best loss: tensor(0.7130, grad_fn=<DivBackward0>)
The function - backprop - took 10.014525175094604 seconds to complete
The function - post_backprop - has just started at 1595137389.65364
The function - post_backprop - took 4.38690185546875e-05 seconds to complete
The function - evaluate fitness - has just started at 1595137389.6537302
{40: <neat.genome.DefaultGenome object at 0x7fe1fb4c07c0>, 26: <neat.genome.DefaultGenome object at 0x7fe1fb4a9ee0>, 48: <neat.genome.DefaultGenome object at 0x7fe1fb4c6c10>, 49: <neat.genome.DefaultGenome object at 0x7fe1fb4c6fa0>, 50: <neat.genome.DefaultGenome object at 0x7fe1fb4c70d0>}
The function - evaluate fitness - took 0.018066883087158203 seconds to complete
The function - post evaluate - has just started at 1595137389.6720839
Population's average fitness: 0.63402 stdev: 0.59881
Best fitness: 1.38097 - size: (2, 2) - species 1 - id 40
Key: 40
Fitness: 1.3809747695922852


mean improvement: 0.0
best improvement: tensor(0., grad_fn=<SubBackward0>)
best loss: tensor(0.7007, grad_fn=<DivBackward0>)
The function - backprop - took 7.109059810638428 seconds to complete
The function - post_backprop - has just started at 1595137412.030468
The function - post_backprop - took 3.4809112548828125e-05 seconds to complete
The function - evaluate fitness - has just started at 1595137412.030535
{51: <neat.genome.DefaultGenome object at 0x7fe1fb41f9d0>, 40: <neat.genome.DefaultGenome object at 0x7fe1fb4c07c0>, 57: <neat.genome.DefaultGenome object at 0x7fe1fb4ccf10>, 58: <neat.genome.DefaultGenome object at 0x7fe1fb4d10d0>, 59: <neat.genome.DefaultGenome object at 0x7fe1fb4d1490>}
The function - evaluate fitness - took 0.02182602882385254 seconds to complete
The function - post evaluate - has just started at 1595137412.0524611
Population's average fitness: 1.05966 stdev: 0.50294
Best fitness: 1.40185 - size: (2, 3) - species 1 - id 51
Key: 51
Fitness: 1.4018508195877075


In [ ]:
len(xor_outputs)

In [ ]:
len(preds)

In [12]:
g

In [16]:
print(g)

Key: 61
Fitness: 1.4394285678863525
Nodes:
	0 DefaultNodeGene(key=0, bias=-0.18528392910957336, response=1.0, activation=sigmoid, aggregation=sum)
	6 DefaultNodeGene(key=6, bias=1.5382388830184937, response=1.0, activation=sigmoid, aggregation=sum)
Connections:
	DefaultConnectionGene(key=(-2, 0), weight=0.8921303153038025, enabled=False)
	DefaultConnectionGene(key=(-1, 6), weight=-0.4905124306678772, enabled=True)
	DefaultConnectionGene(key=(6, 0), weight=0.2004733681678772, enabled=True)


In [17]:
g.add_connection()

TypeError: add_connection() missing 5 required positional arguments: 'config', 'input_key', 'output_key', 'weight', and 'enabled'

In [18]:
h = deepcopy(g)

In [21]:
h.mutate_add_node(config)

AttributeError: 'Config' object has no attribute 'get_new_node_key'

In [22]:
print(h)

Key: 61
Fitness: 1.4394285678863525
Nodes:
	0 DefaultNodeGene(key=0, bias=-0.18528392910957336, response=1.0, activation=sigmoid, aggregation=sum)
	6 DefaultNodeGene(key=6, bias=1.5382388830184937, response=1.0, activation=sigmoid, aggregation=sum)
Connections:
	DefaultConnectionGene(key=(-2, 0), weight=0.8921303153038025, enabled=False)
	DefaultConnectionGene(key=(-1, 6), weight=-0.4905124306678772, enabled=True)
	DefaultConnectionGene(key=(6, 0), weight=0.2004733681678772, enabled=True)


In [31]:
h.mutate_add_node(p.config.genome_config)

In [34]:
h.mutate_delete_connection()

In [35]:
print(h)

Key: 61
Fitness: 1.4394285678863525
Nodes:
	0 DefaultNodeGene(key=0, bias=-0.18528392910957336, response=1.0, activation=sigmoid, aggregation=sum)
	6 DefaultNodeGene(key=6, bias=1.5382388830184937, response=1.0, activation=sigmoid, aggregation=sum)
	16 DefaultNodeGene(key=16, bias=-0.8723593067338321, response=1.0, activation=sigmoid, aggregation=sum)
	17 DefaultNodeGene(key=17, bias=0.37747336865149217, response=1.0, activation=sigmoid, aggregation=sum)
Connections:
	DefaultConnectionGene(key=(-2, 16), weight=1.0, enabled=True)
	DefaultConnectionGene(key=(-2, 17), weight=1.0, enabled=True)
	DefaultConnectionGene(key=(-1, 6), weight=-0.4905124306678772, enabled=True)
	DefaultConnectionGene(key=(6, 0), weight=0.2004733681678772, enabled=True)
	DefaultConnectionGene(key=(16, 0), weight=0.8921303153038025, enabled=True)


In [37]:
h.add_connection(p.config.genome_config, 16, 17, 0.2, True)
h.add_connection(p.config.genome_config, 17, 0, 0.2, True)

In [38]:
print(h)

Key: 61
Fitness: 1.4394285678863525
Nodes:
	0 DefaultNodeGene(key=0, bias=-0.18528392910957336, response=1.0, activation=sigmoid, aggregation=sum)
	6 DefaultNodeGene(key=6, bias=1.5382388830184937, response=1.0, activation=sigmoid, aggregation=sum)
	16 DefaultNodeGene(key=16, bias=-0.8723593067338321, response=1.0, activation=sigmoid, aggregation=sum)
	17 DefaultNodeGene(key=17, bias=0.37747336865149217, response=1.0, activation=sigmoid, aggregation=sum)
Connections:
	DefaultConnectionGene(key=(-2, 16), weight=1.0, enabled=True)
	DefaultConnectionGene(key=(-2, 17), weight=1.0, enabled=True)
	DefaultConnectionGene(key=(-1, 6), weight=-0.4905124306678772, enabled=True)
	DefaultConnectionGene(key=(6, 0), weight=0.2004733681678772, enabled=True)
	DefaultConnectionGene(key=(16, 0), weight=0.8921303153038025, enabled=True)
	DefaultConnectionGene(key=(16, 17), weight=0.2, enabled=True)
	DefaultConnectionGene(key=(17, 0), weight=0.2, enabled=True)


In [39]:
p.config.genome_config.input_keys

[-1, -2]

In [40]:
h.nodes

{0: <neat.genes.DefaultNodeGene at 0x7fe1fd9bbc70>,
 6: <neat.genes.DefaultNodeGene at 0x7fe1fcbfb8b0>,
 16: <neat.genes.DefaultNodeGene at 0x7fe1fd351280>,
 17: <neat.genes.DefaultNodeGene at 0x7fe1fceef850>}

In [200]:
node_tracker = {node_id:{'depth':0, 'output_ids':[]} for node_id in h.nodes}
for node_id in p.config.genome_config.input_keys:
    node_tracker[node_id] = {'depth':0, 'output_ids':[]}
trace_stack = [node_id for node_id in p.config.genome_config.input_keys]

In [201]:
trace_stack

[-1, -2]

In [202]:
node_tracker

{0: {'depth': 0, 'output_ids': []},
 6: {'depth': 0, 'output_ids': []},
 16: {'depth': 0, 'output_ids': []},
 17: {'depth': 0, 'output_ids': []},
 -1: {'depth': 0, 'output_ids': []},
 -2: {'depth': 0, 'output_ids': []}}

In [203]:
for connection in h.connections:
    print(connection)
    node_tracker[connection[0]]['output_ids'].append(connection[1])

(6, 0)
(-1, 6)
(-2, 16)
(16, 0)
(-2, 17)
(16, 17)
(17, 0)


In [204]:
while len(trace_stack) > 0:
    trace = trace_stack[0]
    my_depth = node_tracker[trace]['depth']
    next_depth = my_depth + 1
    for output_id in node_tracker[trace]['output_ids']:
        node_tracker[output_id]['depth'] = max(node_tracker[output_id]['depth'], next_depth)
        trace_stack.append(output_id)
    del(trace_stack[0])

In [205]:
node_tracker

{0: {'depth': 3, 'output_ids': []},
 6: {'depth': 1, 'output_ids': [0]},
 16: {'depth': 1, 'output_ids': [0, 17]},
 17: {'depth': 2, 'output_ids': [0]},
 -1: {'depth': 0, 'output_ids': [6]},
 -2: {'depth': 0, 'output_ids': [16, 17]}}

In [206]:
for node_id, node in node_tracker.items():
    node['output_layers']=[]
    node['needs_skip'] = False
    node['id'] = node_id
    for output_id in node['output_ids']:
        node['output_layers'].append(node_tracker[output_id]['depth'])
        if node_tracker[output_id]['depth'] > (node['depth']+1):
            node['needs_skip'] = True

In [207]:
node_tracker

{0: {'depth': 3,
  'output_ids': [],
  'output_layers': [],
  'needs_skip': False,
  'id': 0},
 6: {'depth': 1,
  'output_ids': [0],
  'output_layers': [3],
  'needs_skip': True,
  'id': 6},
 16: {'depth': 1,
  'output_ids': [0, 17],
  'output_layers': [3, 2],
  'needs_skip': True,
  'id': 16},
 17: {'depth': 2,
  'output_ids': [0],
  'output_layers': [3],
  'needs_skip': False,
  'id': 17},
 -1: {'depth': 0,
  'output_ids': [6],
  'output_layers': [1],
  'needs_skip': False,
  'id': -1},
 -2: {'depth': 0,
  'output_ids': [16, 17],
  'output_layers': [1, 2],
  'needs_skip': True,
  'id': -2}}

In [230]:
layers = {}
for node_id, node in node_tracker.items():
    if not node['depth'] in layers:
        layers[node['depth']] = {
            'nodes':{node_id:node}
        }
    else:
        layers[node['depth']]['nodes'][node_id] = node
for layer_id, layer in layers.items():
    if 0 in layer['nodes']:
        layer['is_output_layer'] = True
    else:
        layer['is_output_layer'] = False
    layer_index = 0
    biases = []
    if layer['is_output_layer']:
        layer['out_weights'] = []
    else:
        layer['out_weights'] = [[0 for __ in layers[layer_id+1]['nodes']] for _ in layer['nodes']]
                       
    for node_id, node in layer['nodes'].items():
        node['layer_index'] = layer_index
        layer_index += 1
        if node['id'] < 0:
            biases.append(1)
        else:
            biases.append(h.nodes[node['id']].bias)
        for output_id in node['output_ids']:
            if (node['id'], output_id) in h.connections:
                if output_id in layers[layer_id + 1]['nodes']:
                    layer['out_weights'][node['layer_index']][layers[layer_id + 1]['nodes'][output_id]['layer_index']] = h.connections[(node['id'], output_id)].weight
                else:
                    # TODO: this is a skip layer to deal with
                    pass
            else:
                # this is not a connection that exists, so default of 0 holds
                pass
            
    layer['out_tensor'] = torch.tensor(layer['out_weights'])
    layer['out_tensor_shape'] = layer['out_tensor'].shape

In [231]:
layers

{3: {'nodes': {0: {'depth': 3,
    'output_ids': [],
    'output_layers': [],
    'needs_skip': False,
    'id': 0,
    'layer_index': 0}},
  'is_output_layer': True,
  'out_weights': [],
  'out_tensor': tensor([]),
  'out_tensor_shape': torch.Size([0])},
 1: {'nodes': {6: {'depth': 1,
    'output_ids': [0],
    'output_layers': [3],
    'needs_skip': True,
    'id': 6,
    'layer_index': 0},
   16: {'depth': 1,
    'output_ids': [0, 17],
    'output_layers': [3, 2],
    'needs_skip': True,
    'id': 16,
    'layer_index': 1}},
  'is_output_layer': False,
  'out_weights': [[0], [0.2]],
  'out_tensor': tensor([[0.0000],
          [0.2000]]),
  'out_tensor_shape': torch.Size([2, 1])},
 2: {'nodes': {17: {'depth': 2,
    'output_ids': [0],
    'output_layers': [3],
    'needs_skip': False,
    'id': 17,
    'layer_index': 0}},
  'is_output_layer': False,
  'out_weights': [[0.2]],
  'out_tensor': tensor([[0.2000]]),
  'out_tensor_shape': torch.Size([1, 1])},
 0: {'nodes': {-1: {'depth': 0,

In [239]:
a = torch.randn(2,3)
b = torch.randn(3, 1)
c = torch.matmul(a, b)

In [190]:
layer_id

0

In [191]:
out_weights

[-0.4905124306678772, 1.0]

In [164]:
h.nodes[6].bias

1.5382388830184937

In [160]:
layers

{3: {'nodes': [{'depth': 3,
    'output_ids': [],
    'output_layers': [],
    'needs_skip': False,
    'id': 0,
    'layer_index': 0}]},
 1: {'nodes': [{'depth': 1,
    'output_ids': [0],
    'output_layers': [3],
    'needs_skip': True,
    'id': 6,
    'layer_index': 0},
   {'depth': 1,
    'output_ids': [0, 17],
    'output_layers': [3, 2],
    'needs_skip': True,
    'id': 16,
    'layer_index': 1}]},
 2: {'nodes': [{'depth': 2,
    'output_ids': [0],
    'output_layers': [3],
    'needs_skip': False,
    'id': 17,
    'layer_index': 0}]},
 0: {'nodes': [{'depth': 0,
    'output_ids': [6],
    'output_layers': [1],
    'needs_skip': False,
    'id': -1,
    'layer_index': 0},
   {'depth': 0,
    'output_ids': [16, 17],
    'output_layers': [1, 2],
    'needs_skip': True,
    'id': -2,
    'layer_index': 1}]}}

In [158]:
print(h)

Key: 61
Fitness: 1.4394285678863525
Nodes:
	0 DefaultNodeGene(key=0, bias=-0.18528392910957336, response=1.0, activation=sigmoid, aggregation=sum)
	6 DefaultNodeGene(key=6, bias=1.5382388830184937, response=1.0, activation=sigmoid, aggregation=sum)
	16 DefaultNodeGene(key=16, bias=-0.8723593067338321, response=1.0, activation=sigmoid, aggregation=sum)
	17 DefaultNodeGene(key=17, bias=0.37747336865149217, response=1.0, activation=sigmoid, aggregation=sum)
Connections:
	DefaultConnectionGene(key=(-2, 16), weight=1.0, enabled=True)
	DefaultConnectionGene(key=(-2, 17), weight=1.0, enabled=True)
	DefaultConnectionGene(key=(-1, 6), weight=-0.4905124306678772, enabled=True)
	DefaultConnectionGene(key=(6, 0), weight=0.2004733681678772, enabled=True)
	DefaultConnectionGene(key=(16, 0), weight=0.8921303153038025, enabled=True)
	DefaultConnectionGene(key=(16, 17), weight=0.2, enabled=True)
	DefaultConnectionGene(key=(17, 0), weight=0.2, enabled=True)
